<a href="https://colab.research.google.com/github/iggyloh/Tesla-and-amazon-stock-dashboard/blob/main/DAC_proj_2_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from IPython.display import  display
import pandas as pd
from typing import TypedDict, Annotated
import sqlite3

from langchain import hub
from langchain_core.prompts import MessagesPlaceholder
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.text_splitter  import RecursiveCharacterTextSplitter
from langgraph.graph.state import StateGraph, START, END
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

from langchain.schema import BaseMessage
from langchain_community.vectorstores import Chroma, InMemoryVectorStore
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_history_aware_retriever
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

from IPython.display import Image, display
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

import gradio as gr

c:\Users\script.pc #93\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Loading necessary APIs and files

### 1.1. Loading APIs and connecting embedding model

In [ ]:
# Load API env
load_dotenv('api.env')

# Langsmith tracing
trace = os.getenv("LANGCHAIN_TRACING_V2")
langsmith = os.getenv("LANGCHAIN_API_KEY")


# OpenAi model
gpt = ChatOpenAI(
    model = 'gpt-3.5-turbo',
    temperature = 0.1)


# Embedding model
embedding_model = OpenAIEmbeddings(
    model = 'text-embedding-ada-002',
    openai_api_key = os.getenv('OPENAI_API_KEY')
)


# Small test
try:
    response = gpt.invoke('Which player won the most league of legends world championships?')
    print(response.content)
except Exception as e:
    print(f"Error {e}")

C:\Users\script.pc #93\AppData\Local\Temp\ipykernel_17212\2742753873.py:16: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(


Lee "Faker" Sang-hyeok from South Korea has won the most League of Legends World Championships, with a total of three titles. He won the championships in 2013, 2015, and 2016 with his team, SK Telecom T1.


### 1.2. Loading and cleaning papers.csv
We collect only the column 'full_text' and turn it into a pandas series.

In [3]:
# Print and display csv
df = pd.read_csv('papers.csv')
print("--- This is the raw CSV: ---")
display(df.head())


# Due to the pressence of 3 NA entries, we fill them with " "
texts = df['full_text'].fillna(" ")


# To save on cost and use this as a POC/MVP we only keep 10 entries (randomly samplied)
texts = texts.sample(n=10, random_state=69)
print("--- This is the pandas series of ['full_texts'] randomly sampling 10 papers: ---")
display(texts)

--- This is the raw CSV: ---


,source_id,year,title,abstract,full_text
0,27,1987,Bit-Serial Neural Networks,NaN,573 \n\nBIT - SERIAL NEURAL NETWORKS \n\nAlan...
1,63,1987,Connectivity Versus Entropy,NaN,1 \n\nCONNECTIVITY VERSUS ENTROPY \n\nYaser S...
2,60,1987,The Hopfield Model with Multi-Level Neurons,NaN,278 \n\nTHE HOPFIELD MODEL WITH MUL TI-LEVEL N...
3,59,1987,How Neural Nets Work,NaN,442 \n\nAlan Lapedes \nRobert Farber \n\nThe...
4,69,1987,Spatial Organization of Neural Networks: A Pro...,NaN,740 \n\nSPATIAL ORGANIZATION OF NEURAL NEn...


--- This is the pandas series of ['full_texts'] randomly sampling 10 papers: ---


8391    Robust Attribution Regularization\n\nJiefeng C...
3068    Information Bottleneck Optimization and\n\nInd...
7231    EEG-GRAPH: A Factor-Graph-Based Model for\nCap...
1965    Spectral Kernel Methods for Clustering \n\nN e...
4704    A Conditional Multinomial Mixture Model for\n\...
608     Information,  prediction,  and  query by \n\nc...
5294    Local Decorrelation for Improved Pedestrian De...
9254    Identiﬁcation of Conditional Causal Effects\n\...
9285    Beyond Conﬁdence Regions: Tight Bayesian\n\nAm...
8434    Chirality Nets for Human Pose Regression\n\nRa...
Name: full_text, dtype: object

### 1.3. Defining Parent Graph before RAGS and SQL Graph and memory config for persistence

In [21]:
# Parent graph
class ParentGraph(TypedDict):
    """Represents the state of our graph. Enforcing questions and answers to be dtype string. -spnc"""
    question: str
    question_type: str
    answer: str


# Config for memory saver, only need 1 thread here
# We're defining this early to check if the sql builder works
config = {
    "configurable": {
        'thread_id': '1',                  
        'checkpoint_ns': 'checkpoint',      
        'checkpoint_id': '13376969god'       
    }
}

memory = MemorySaver() 

## 2. Building the RAGS and RAGS Graph

### 2.1. Embedding each research paper and storing in a vector store

In [22]:
# Object "vector_store" will hold each research paper and their corresponding embedded values
vector_store = InMemoryVectorStore(embedding_model)


# Turning pandas series into document objects
documents = [Document(page_content=text) for text in texts]


# Chunking texts into smaller sizes
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = [Document(page_content=chunk) for doc in documents for chunk in text_splitter.split_text(doc.page_content)]


# Generate embeddings for each chunk
embeddings = embedding_model.embed_documents([doc.page_content for doc in all_splits])


# Store and embed in vector store simultaneously
document_ids = vector_store.add_documents( # Can be ran again later to add more research papers yeah
    documents=all_splits,
    embeddings=embeddings 
)


# Verifying embedding done and document IDs
print(f"First 3 document IDs: {document_ids[:3]}")
print(f"First 3 document contents: {[doc.page_content for doc in all_splits[:3]]}")
print(f"Number of embeddings: {len(embeddings)}")
print(f"Shape of each embedding (dimensions): {len(embeddings[0])}") 


First 3 document IDs: ['1a2ea0be-1dd4-49f6-965d-48ae33f6395e', '244feca5-2962-47b8-a5c9-232a2dbec1b1', 'e0055cfc-b309-441f-8600-99f060c0c344']
First 3 document contents: ['Robust Attribution Regularization\n\nJiefeng Chen ⇤ 1 Xi Wu ⇤ 2 Vaibhav Rastogi †2\n\n1 University of Wisconsin-Madison\n\nYingyu Liang 1\n\nSomesh Jha 1,3\n\n2 Google\n\n3 XaiPient\n\nAbstract', 'An emerging problem in trustworthy machine learning is to train models that pro-\nduce robust interpretations for their predictions. We take a step towards solving\nthis problem through the lens of axiomatic attribution of neural networks. Our\ntheory is grounded in the recent work, Integrated Gradients (IG) [STY17], in\naxiomatically attributing a neural network’s output change to its input change.\nWe propose training objectives in classic robust optimization models to achieve\nrobust IG attributions. Our objectives give principled generalizations of previous\nobjectives designed for robust predictions, and they naturally

### 2.2 Testing retrieval from the vector store

In [23]:
# Golden retriever from the vector store, who's a good boy?
retriever = vector_store.as_retriever()


# RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=gpt, 
    chain_type="stuff",  # 'stuff' means the full document is fed to the model, not a restaurant lolol
    retriever=retriever
)


# Example query, if the RAGS works, we should be quoting from one of the papers we fed it
query = "What is backpropogation? Quote the paper you are retrieving from."


# Use the QA chain to answer the query based on retrieved documents
response = qa_chain.run(query)
print(response)


# Retrieve documents manually, check if the mentioned paper is within the results (spoiler: yes it works)
results = retriever.get_relevant_documents("What is backpropogation?")
print(f"Retrieved documents: {results}")

Backpropagation is a learning algorithm for continually running fully recurrent neural networks. This information is from the paper "A learning algorithm for continually running fully recurrent neural networks" by R. J. Williams and D. Zipser in Neural Computation, 1989.
Retrieved documents: [Document(id='a949ef01-b618-4d37-9c75-519125bece58', metadata={}, page_content='in Proceedings of the 16th conference on Uncertainty in artiﬁcial intelligence, 2000, pp. 626–633.\n\n[15] J. S. Yedidia, W. T. Freeman, Y. Weiss et al., “Generalized belief propagation,” in Advances in Neural\n\nInformation Processing Systems, vol. 13, 2000, pp. 689–695.\n\n[16] W. R. Gilks, S. Richardson, and D. Spiegelhalter, Markov chain Monte Carlo in practice. CRC Press,\n\n1995.\n\n[17] S. Chib and E. Greenberg, “Understanding the Metropolis-Hastings algorithm,” The American Statisti-\n\ncian, vol. 49, no. 4, pp. 327–335, 1995.\n\n[18] V. Kolmogorov and R. Zabin, “What energy functions can be minimized via graph 

### 2.3. RAG SubGraph

In [24]:
# Class states that ensures questions and answers are string
class RagState(TypedDict):
    """Represents the state of our graph. Enforcing questions and answers to be dtype string. -spnc"""
    question: str
    answer: str


# RAG SubGraph to be called if it is a question answerable by RAG
def rag_subgraph(state: ParentGraph):
    """Handle RAG-specific questions."""
    question = state["question"]

    # Retieving done like in the above testing cell
    search_results = retriever.get_relevant_documents(question)
    
    # Combine all retrieved documents for MAXIMUM CONTEXT
    context = "\n".join([doc.page_content for doc in search_results])
    
    response = qa_chain.run(question + "\n" + context)  

    # If we can't answer with the RAG then we won't have an answer I guess, ask a better question T_T
    answer = response if response else "No response available from training, ask a question relating to Deep Learning research in the past 30 years."
    
    return {"answer": answer}


## 3. Building the SQL Graph from 'main.db' generated by 'sql-data-exploration-database-creation.ipynb'

### 3.1. Defining our own prompt template because we are gangster like that

In [25]:
# Pulling prompt template form LangChain
query_prompt_template = PromptTemplate(
    input_variables=["dialect", "top_k", "table_info", "input"],
    template="""
    You are a SQL query generator. Given the following information:
    - Database dialect: {dialect}
    - Tables: {table_info}
    - Question: {input}

    Generate the best SQL query to answer the question.
    Provide only the query, and make sure it is syntactically valid.
    The fields are full_name from combined_df if they mention about author names.

    You can do your own descriptive statistics like average papers each year etc.
    """
)

### 3.2. SQL SubGraph built on functions to execute syntatically correct queries

In [55]:
# Class states that ensures questions and answers are string
class SqlState(TypedDict):
    """Represents the state of our graph. Enforcing questions and answers to be dtype string. -spnc"""
    question: str
    query: str
    result: str
    answer: str


# Class states to ensure query output is string and eventually syntatically correct
class QueryOutput(TypedDict):
    """Generated SQL query."""
    query: Annotated[str, ..., "Syntactically valid SQL query."]


# Function to get ALL table infos 
def get_table_info():
    """Return information about the tables in the database."""
    conn = sqlite3.connect('main.db')
    cursor = conn.cursor()
    
    # Query to get all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    # Optionally, get more detailed info if needed
    conn.close()
    
    return tables


# Function to write syntatically correct SQLite3 queries
def write_query(state: SqlState):
    """Generate SQL query to fetch information."""
    
    # Open the SQLite connection here
    conn = sqlite3.connect('main.db')
    cursor = conn.cursor()
    
    # Your code to generate the SQL query based on the question
    prompt = query_prompt_template.invoke(
        {
            "dialect": "sqlite",  
            "top_k": 5,
            "table_info": get_table_info(), 
            "input": state["question"],
        }
    )
    
    structured_llm = gpt.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    
    # Optionally, you could run the query directly here if needed for debugging
    generated_query = result["query"]
    
    # Return the query string
    conn.close() 
    return {"query": generated_query}


# Function to execute the query generated from write_query function
def execute_query(state: SqlState):
    """Execute SQL query."""
    
    # Open the SQLite connection here
    conn = sqlite3.connect('main.db')
    cursor = conn.cursor()
    
    query = state["query"]
    
    # Execute the query and fetch the result
    cursor.execute(query)
    result = cursor.fetchall()  # Assuming you want to fetch all rows
    
    # Close the connection
    conn.close()
    
    # Return the result (you can format or process it as needed)
    return {"result": result}


# Function to generate answer based on SQL query results
def generate_answer(state: SqlState):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'
    )
    response = gpt.invoke(prompt)
    return {"answer": response.content}


# SQL SubGraph!
def sql_subgraph(state: SqlState):
    """Handle SQL-based questions using the sql_builder."""
    
    # Just invoke the sql_builder with the current state
    result = sql.invoke({"question": state["question"]}, config=config)
    
    # Return the answer generated by the workflow
    return {"answer": result["answer"]}

### 3.3. SQL Builder to finish the sequence and test the builder

In [27]:
# Create the state graph to define the workflow
sql_builder = StateGraph(SqlState).add_sequence(
  [write_query, execute_query, generate_answer]
)


# Add the starting edge to the workflow
sql_builder.add_edge(START, "write_query")


# Compile the workflow
sql = sql_builder.compile(checkpointer=memory)


# Example query
result = sql.invoke({"question": "How many papers did Michael Jordan write?"}, config=config)


# Print the result
print(result["answer"])

Answer: Michael Jordan wrote a total of 113 papers.


## 4. Determining if a question should be answered using RAG or our SQL database of inferential/exploratory statistics

### 4.1. Question Classifier and Router
Prompt engineering needs to be further perfected.

In [56]:
# Question classifier
def classify_question(state: ParentGraph):
    """Classify the type of question: SQL or RAG."""
    question = state["question"]

    classification_prompt = f"""
    You are a classifier. Determine the appropriate category for the following question:
    - "SQL" if the question is related to SQL queries, databases, retrieving structured data, or performing data analysis using SQL or relational database concepts. Mainly if the user asks about what data you are mainly working from, and the statistics from this data such as "who wrote the most papers?" or "how many papers did <full_name> write?".
    - "RAG" if the question involves retrieving information from the research papers, combining data from multiple sources, or problem-solving using Retrieval-Augmented Generation (RAG) techniques.

    Question: {question}
    Answer with only "SQL" or "RAG".
    """

    question_type = gpt.invoke(classification_prompt).content.strip()

    return {"question_type": question_type, "question": question}


# Router to pick RAG/SQL to answer
def route_based_on_question_type(state: ParentGraph):
    if state["question_type"] == "SQL":
        return "sql_subgraph"
    elif state["question_type"] == "RAG":
        return "rag_subgraph"
    else:
        return END

### 4.2. Finalizing "nodes" to the graph functions with a conditional "edge"

In [57]:
# Calling parent graph function to determine state/class of query/answer
parent_graph = StateGraph(ParentGraph)


# Add nodes to the graph
parent_graph.add_node("classify_question", classify_question)
parent_graph.add_node("sql_subgraph", sql_subgraph)
parent_graph.add_node("rag_subgraph", rag_subgraph)


# Conditional "edge" for routing based on classification
parent_graph.add_conditional_edges("classify_question", route_based_on_question_type)


# Starting "edge"
parent_graph.add_edge(START, "classify_question")

## 5. Compiling all graph functions to generate results

In [58]:
# Memory saver to save states for persistence, checkpointing at every super-step
memory = MemorySaver()
parent = parent_graph.compile(checkpointer=memory)


# Example query
query = "How many papers did Michael Jordan write??"
result = parent.invoke({"question": query}, config= config)


# Moment of truth.....
answer = result["answer"]
print("Answer:", answer)

Answer: The SQL query returned a result of 113 papers written by Michael Jordan.


### 6. Gradio Interface!
spnc: i have no idea what this part does so i can't annotate this, astrid and dr chat giga carried this....

In [62]:
import gradio as gr

# Store conversation history for display in the UI
conversation_history = []

def process_question(user_input):
    global memory  # Keep track of memory state, but idk how to stop it from persisting and answering previous questions yet
    
    # Add the user's question to the history
    conversation_history.append(f"User: {user_input}")

    # Combine all conversation history to provide context
    conversation_context = "\n".join(conversation_history)

    try:
        # Send the conversation context along with the new question and use memory for persistence
        result = parent.invoke({"question": conversation_context}, config=config)
        answer_text = result["answer"]

        # Add the model's answer to the history
        conversation_history.append(f"Model: {answer_text}")
        
        # Return the answer
        return answer_text
    except Exception as e:
        return f"Error processing your question: {str(e)}"
    

banner_url = 'https://img.freepik.com/premium-vector/classroom-vector-illustration-back-school-after-holiday-cartoon-classroom_1020331-7709.jpg?w=1060'


# Create the Gradio interface
with gr.Blocks() as iface:
    # Add the banner image
    gr.Markdown(f"""
    <div style="text-align: center; margin-bottom: 20px;">
        <img src="{banner_url}" alt="Professor John 'Master Splinter' Yeow" style="max-width: 100%; height: auto;">
    </div>
    """)

    # Title and description
    gr.Markdown("""
    ## Chat with Professor John 'Master Splinter' Yeow's Q&A System
    You can ask questions, and the system will remember previous conversation context.
    """)

    # User input for the question
    with gr.Row():
        question_input = gr.Textbox(
            label="Ask Your Question", lines=2, placeholder="Enter your question here..."
        )
    
    # Output for the answer
    with gr.Row():
        answer_output = gr.Textbox(label="Answer", lines=8)

    # Add the submit button
    submit_btn = gr.Button("Submit")
    
    # When the submit button is clicked, process the question and display the answer
    submit_btn.click(
        fn=process_question,
        inputs=question_input,
        outputs=answer_output
    )

# Launch the Gradio app
if __name__ == "__main__":
    iface.launch(debug=True)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


## Testing and debugging

In [ ]:

# retriever = vector_store.as_retriever()
# # Initialize the RetrievalQA chain
# qa_chain = RetrievalQA.from_chain_type(
#     llm=gpt, 
#     chain_type="stuff",  # 'stuff' means the full document is fed to the model
#     retriever=retriever
# )

# # Example query
# query = "What is backpropogation? Quote the paper you are retrieving from."


# # Use the QA chain to answer the query based on retrieved documents
# response = qa_chain.run(query)

# print(response)

# # Retrieve documents manually
# results = retriever.get_relevant_documents("What is backpropogation?")
# print(f"Retrieved documents: {results}")

Backpropagation is a learning algorithm for continually running fully recurrent neural networks. This information is from the paper "A learning algorithm for continually running fully recurrent neural networks" by R. J. Williams and D. Zipser in Neural Computation, 1989.
Retrieved documents: [Document(id='a949ef01-b618-4d37-9c75-519125bece58', metadata={}, page_content='in Proceedings of the 16th conference on Uncertainty in artiﬁcial intelligence, 2000, pp. 626–633.\n\n[15] J. S. Yedidia, W. T. Freeman, Y. Weiss et al., “Generalized belief propagation,” in Advances in Neural\n\nInformation Processing Systems, vol. 13, 2000, pp. 689–695.\n\n[16] W. R. Gilks, S. Richardson, and D. Spiegelhalter, Markov chain Monte Carlo in practice. CRC Press,\n\n1995.\n\n[17] S. Chib and E. Greenberg, “Understanding the Metropolis-Hastings algorithm,” The American Statisti-\n\ncian, vol. 49, no. 4, pp. 327–335, 1995.\n\n[18] V. Kolmogorov and R. Zabin, “What energy functions can be minimized via graph 

In [32]:
# query = "What is backpropagation?"
# result = qa_chain.run(query)
# print(result)


Backpropagation is a common method used in training artificial neural networks. It involves calculating the gradient of the loss function with respect to the weights of the network, which allows for adjusting the weights in a way that minimizes the error in the network's output.
